
### Prerequisites
Open two terminals and setup these tools in both of them:
- Install AWS CLI and Node using

```bash
source setup_node.sh
```
- EB CLI as:

```bash
cd
python -m pip install virtualenv
git clone https://github.com/aws/aws-elastic-beanstalk-cli-setup.git
python ./aws-elastic-beanstalk-cli-setup/scripts/ebcli_installer.py
echo 'export PATH="/root/.ebcli-virtual-env/executables:$PATH"' >> ~/.bash_profile && source ~/.bash_profile
eb --version
```



In order to complete the project you will need to do the following:
### 1. Provision the AWS Resources
Provision the necessary AWS services your app needs:
- S3 bucket with suitable access policy and ACL policy
- RDS instance with public access
- Update the **udagram/set_env.sh** file with the right values. 



### 2. Clone a Github repo 
Clone the <a href="https://github.com/udacity/nd0067-c4-deployment-process-project-starter" target="_blank">GitHub repo</a> or create a new repo using the code present in the **/home/workspace/udagram/** directory. 
```bash
git clone https://github.com/[username]/nd0067-c4-deployment-process-project-starter.git
cd nd0067-c4-deployment-process-project-starter/udagram
```
You can connect the workspace with the Github repo using the SSH keys.


### 3. Local Run
Let's run the backend API first:
```bash
cd udagram/
source set_env.sh
cd udagram-api/
rm -rf node_modules/
rm -rf package-lock.json  
# Install the package dependencies, and ignore the warnings
npm install .
# Generate the "www/" folder contaning the autogenerated files.
npx tsc
npm run start
# If everything works fine with the "npm run start", build the files. 
npm run dev 
```
Elastic beanstalk will run "npm run start" command by default. Ideally, running the "npx tsc" command would generate the "www/" folder contaning the autogenerated files.

Next, run the frontend code as:

```bash
cd udagram/
source set_env.sh
cd udagram-frontend/ 
rm -rf node_modules/
rm -rf package-lock.json 
npm install 
npm run start
```
Click on the OPEN APP button below to open the frontend application in a new browser tab. 




<button id="ulab-button-b3f3478d" class="ulab-btn--primary"></button>


### 4. Deploy the Backend API Manually
Navigate to the API directory and run these commands:

```bash
cd udagram/udagram-api
# The "eb init" command will create ".elasticbeanstalk/config.yml" file in the current directory. 
# Use the node.js 12 or 14 and the default region as applicable to you
eb init
# The "eb create" command will create an EC2 instance, an S3 bucket to store your source code, logs, and other artifacts, and other AWS resources
eb create --single --keyname mykeypair --instance-types t2.medium
```

Once you application is deployed to the Elastic Beanstalk, it will automatically run these comamnds on your behalf:

```bash
npm install 
npm start
```

**Important**: Specify the environment variables in the **Elastic Beanstalk environment >> Configuration >> Software settings >> Environment Properties** section. Beanstalk will pass these variables to the EC2 instace. You can save all **set_env.sh** variables into the Beanstalk. 


### 5. Continuous Integration and Continuous Deployment
Configure a CircleCi pipeline to automate your deployments. Set up a CircleCI [free account](https://circleci.com/signup/?source-button=free), if you haven't already. 


We have provided you with a sample **.circleci/config.yml** to get started. You have to update the `build` and `deploy` jobs. The `workflows` section is already complete.  


We have marked the individual steps that need your updates with **"TODO" comments**. 


**Important**: The CircleCI cannot read your local environment variables. Therefore, you have to save save all **set_env.sh** variables into the CircleCI's project settings



To make the CircleCI config commands work, you may have to update the **Backend** *udagram/udagram-api/package.json* file:

```json
"scripts": {
    "start": "node ./www/server.js",
    "tsc": "npx tsc",
    "dev": "npx ts-node-dev --respawn --transpile-only ./src/server.ts",
    "prod": "npx tsc && node ./www/server.js",
    "clean": "rm -rf www/ || true",
    "deploy": "npm run build && eb list && eb use udagram-api-dev && eb deploy",
    "build": "npm install . && npm run clean && tsc && cp -rf src/config www/config && cp -R .elasticbeanstalk www/.elasticbeanstalk && cp .npmrc www/.npmrc && cp package.json www/package.json && cd www && zip -r Archive.zip . && cd ..",
    "test": "echo \"Error: no test specified\" && exit 1"
  },
```



You will have to update the following **Frontend** files in the frontend directory

1. **/udagram/udagram-frontend/bin/deploy.sh** - Your CircleCI uses this files on one of the stages. Update this file with the S3 bucket you've created manually at the beginning of this project.



2. **/udagram/udagram-frontend/src/environments/environment.ts** - Update the `apiHost` variable in this file to point to the backend API hosted on Elastic Beanstalk. Here is an example how it would look like:

```bash
# Before (for local run)
apiHost: 'http://localhost:8080/api/v0'
# After (when your backend API is hosted on Elastic Beanstalk)
apiHost: 'https://udagram-api-dev-75139724085.us-east-1.elasticbeanstalk.com/api/v0'
```


3. **/udagram/udagram-frontend/src/environments/environment.prod.ts** <br>(same edits as in the *environment.ts* file)



Once all these steps are complete, you can trigger the pipeline build by pushing the changes to your Github repo.  


### 6. Documentation and Screenshots 
Refer to the *Documentation and Screenshots* page in the classroom instructions. 
 